## This juypter notebook is tartget towards defining alleles 

The idea is that alleles are on corresponding haplotig and a good blast hit.

map all h contigs on their corresponding p alleles. Use the Assembletic '*oriented_coords.csv' file for this purpose.
This needs to be converted to a bed file or a gff file in order to pull out the overlap between gene models and the h_contig using either bedtools or pandas. Check what overlaps can spit out on the other side. 
>Considerations here are what happens if nothing overlaps or what if muliptle overlaps. 
If nothing overlaps it would be good to look if this area is covered by 2x coverage when doing ph mapping.

Do gene blast of p on h with certain cut off. Pull out the best hit that is located on the corresponding haplotig. If that is not there look for best hit on other haplotigs or primary contigs that is not itself.

>Considerations are that p-h parings that are best hits and corresponding haplotig overule all others. There can only be a single hit. 



In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil


In [3]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ASSEMBLETICS_INPATH = os.path.join(BASE_AA_PATH, 'Assembletics')
BLAST_DB = os.path.join(BASE_AA_PATH, 'blast_DB')
OUT_PATH = os.path.join(BASE_AA_PATH, 'allele_analysis')
OUT_tmp = os.path.join(OUT_PATH, 'tmp')
ALLELE_PATH = os.path.join(OUT_PATH, 'alleles')
if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
if not os.path.isdir(OUT_tmp):
    os.mkdir(OUT_tmp)
if not os.path.exists(os.path.join(OUT_PATH, 'alleles')):
    os.mkdir(os.path.join(OUT_PATH, 'alleles'))

In [4]:
#Define your p and h genome and move it into the allele analysis folder
p_genome = 'Pst_104E_v12_p_ctg'
h_genome = 'Pst_104E_v12_h_ctg'
genome_file_suffix = '.genome_file'
for x in (x + '.fa' for x in [p_genome, h_genome]):
    shutil.copy2(BASE_A_PATH+'/'+x, OUT_tmp)

In [5]:
#Define ENV parameters for blast hits and threads used in blast analysis
n_threads = 16
e_value = 1e-3

In [6]:
def att_column_p(x, y, z, a, b):
    '''Generate attribute column of h on p p_gff dataframe.
    '''
    string = 'h_contig=%s;query_start=%s;query_stop=%s;query_length=%s;query_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

In [7]:
def att_column_h(x, y, z, a, b):
    '''Generate attribute column of h on p h_gff dataframe.
    '''
    string = 'p_contig=%s;ref_start=%s;ref_stop=%s;ref_length=%s;ref_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

In [8]:
def gene_contig_subset(df, contig, feature):
    '''Input the gff and subset the feature columne by gene.
        Return data frame subset by contig and gene'''
    tmp_df = df[(df[0].str.contains(contig)) & (df[2]==feature)]
    tmp_df.sort_values(3,inplace=True)
    tmp_df.reset_index(drop=True, inplace=True)
    return tmp_df

In [9]:
def same_contig_blast(x,y):
    '''Function that checks if the blast hit in columne y is on the same contig as the the query sequence in
    column y.
    '''
    q_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', x).group(1)
    if y != 'NaN':
        hit_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', y).group(1)
    else:
        hit_contig = 'NaN'
    if q_contig == hit_contig:
        return True
    else:
        return False

In [10]:
def target_on_maped_haplotig(t_contig, h_contig_overlap):
    '''Simple function that checks if the target contig is in the list of h overlaping contigs'''
    if t_contig == False or h_contig_overlap == False:
        return False  
    else:
        if t_contig in h_contig_overlap:
            return True
        else:
            return False
        

#### Considerations for blast analysis
Do gene blast and protein blast. Initially go off protein blast results. They should be more imformative as it also includes the coding region and the frame.
> Pull this in as a dataframe and filter it down as well to each contig. Do blasts both ways as well.

In [11]:
#generate the blast databases if not already present
os.chdir(BLAST_DB)
blast_dir_content = os.listdir(BLAST_DB)
for x in blast_dir_content:
    if x.endswith('.fa') and ({os.path.isfile(x + e) for e in ['.psq', '.phr', '.pin'] } != {True}\
           and {os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] } != {True} ):

        make_DB_options = ['-in']
        make_DB_options.append(x)
        make_DB_options.append('-dbtype')
        if 'protein' in x:
            make_DB_options.append('prot')
        else:
            make_DB_options.append('nucl')
        make_DB_command = 'makeblastdb %s' % ' '.join(make_DB_options)
        make_DB_stderr = subprocess.check_output(make_DB_command, shell=True, stderr=subprocess.STDOUT)
        print('%s is done!' % make_DB_command)
print("All databases generated and ready to go!")

All databases generated and ready to go!


In [12]:
blast_dict = {}
blast_dict['gene_fa'] = [x for x  in os.listdir(BLAST_DB) if x.endswith('gene.fa') and 'ph_ctg' not in x]
blast_dict['protein_fa'] = [x for x  in os.listdir(BLAST_DB) if x.endswith('protein.fa') and 'ph_ctg' not in x]
blast_stderr_dict = {}
for key in blast_dict.keys():
    for n, query in enumerate(blast_dict[key]):
        tmp_list = blast_dict[key][:]
        tmp_stderr_list = []
        #this loops through all remaining files and does blast against all those
        del tmp_list[n]
        for db in tmp_list:
            blast_options = ['-query']
            blast_options.append(query)
            blast_options.append('-db')


            blast_options.append(db)
            blast_options.append('-outfmt 6')
            blast_options.append('-evalue')
            blast_options.append(str(e_value))
            blast_options.append('-num_threads')
            blast_options.append(str(n_threads))
            blast_options.append('>')
            if 'gene' in query and 'gene' in db:
                out_name_list = [ query.split('.')[0], db.split('.')[0], str(e_value), 'blastn.outfmt6']
                out_name = os.path.join(OUT_PATH,'.'.join(out_name_list))
                blast_options.append(out_name)
                blast_command = 'blastn %s' % ' '.join(blast_options)
            elif 'protein' in query and 'protein' in db:
                out_name_list = [ query.split('.')[0], db.split('.')[0], str(e_value), 'blastp.outfmt6']
                out_name = os.path.join(OUT_PATH,'.'.join(out_name_list))
                blast_options.append(out_name)
                blast_command = 'blastp %s' % ' '.join(blast_options)
            print(blast_command)
            if not os.path.exists(out_name):
                blast_stderr_dict[blast_command] = subprocess.check_output(blast_command, shell=True, stderr=subprocess.STDOUT)
                print("New blast run and done!")
            else:
                blast_stderr_dict[blast_command] = 'Previously done already!'
                print('Previously done already!')

blastp -query Pst_104E_v12_p_ctg.anno.protein.fa -db Pst_104E_v12_h_ctg.anno.protein.fa -outfmt 6 -evalue 0.001 -num_threads 16 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6
Previously done already!
blastp -query Pst_104E_v12_h_ctg.anno.protein.fa -db Pst_104E_v12_p_ctg.anno.protein.fa -outfmt 6 -evalue 0.001 -num_threads 16 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/Pst_104E_v12_h_ctg.Pst_104E_v12_p_ctg.0.001.blastp.outfmt6
Previously done already!
blastn -query Pst_104E_v12_p_ctg.anno.gene.fa -db Pst_104E_v12_h_ctg.anno.gene.fa -outfmt 6 -evalue 0.001 -num_threads 16 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastn.outfmt6
Previously done already!
blastn -query Pst_104E_v12_h_ctg.anno.gene.fa -db Pst_104E_v12_p_ctg.anno.gene.fa -outfmt 6

In [13]:
#make a sequnece length dict for all genes and proteins for which blast was performed
seq_list = []
length_list =[]
for key in blast_dict.keys():
    for file in blast_dict[key]:
        for seq in SeqIO.parse(open(file), 'fasta'):
            seq_list.append(seq.id)
            length_list.append(len(seq.seq))
length_dict = dict(zip(seq_list, length_list))
        

In [14]:
len(length_dict)

60498

In [15]:
#get assembletics folders
ass_folders = [os.path.join(ASSEMBLETICS_INPATH, x) for x in os.listdir(ASSEMBLETICS_INPATH) if x.endswith('_php_8kbp')]
ass_folders.sort()

In [16]:
#read in the gff files
p_gff = pd.read_csv(BASE_A_PATH+'/'+p_genome+'.anno.gff3', sep='\t', header=None)
h_gff = pd.read_csv(BASE_A_PATH+'/'+h_genome+'.anno.gff3', sep='\t', header=None)

In [17]:
#read in the blast df and add QLgth and QCov columns
blast_out_dict = {}
blast_header = ['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery', 'StopQuery', 'StartTarget',\
              'StopTarget', 'e-value','BitScore']
for key in blast_stderr_dict.keys():
    file = key.split('>')[-1][1:]
    tmp_key = file.split('/')[-1]
    tmp_df = pd.read_csv(file, sep='\t', header=None, names=blast_header)
    tmp_df["QLgth"] = tmp_df["Query"].apply(lambda x: length_dict[x])
    tmp_df["QCov"] = tmp_df['AlnLgth']/tmp_df['QLgth']*100
    tmp_df.sort_values(by=['Query', 'e-value','BitScore', ],ascending=[True, True, False], inplace=True)
    #now make sure to add proteins/genes without blast hit to the dataframes
    #check if gene blast or protein blast and pull out respective query file
    if file.split('.')[-2] == 'blastp':
        tmp_blast_seq = [x for x in blast_dict['protein_fa'] if x.startswith(tmp_key.split('.')[0])][0]
    elif file.split('.')[-2] == 'blastn':
        tmp_blast_seq = [x for x in blast_dict['gene_fa'] if x.startswith(tmp_key.split('.')[0])][0]
    tmp_all_blast_seq = []
    #make list off all query sequences
    for seq in SeqIO.parse(os.path.join(BLAST_DB, tmp_blast_seq), 'fasta'):
        tmp_all_blast_seq.append(str(seq.id))
    tmp_all_queries_w_hit = tmp_df["Query"].unique()
    tmp_queries_no_hit = set(tmp_all_blast_seq) - set(tmp_all_queries_w_hit)
    no_hit_list = []
    #loop over the quieres with no hit and make list of list out of them the first element being the query id
    for x in tmp_queries_no_hit:
        NA_list = ['NaN'] * len(tmp_df.columns)
        NA_list[0] = x
        no_hit_list.append(NA_list)
    tmp_no_hit_df = pd.DataFrame(no_hit_list)
    tmp_no_hit_df.columns = tmp_df.columns
    tmp_no_hit_df['QLgth'] = tmp_no_hit_df.Query.apply(lambda x: length_dict[x])
    tmp_df = tmp_df.append(tmp_no_hit_df)
    tmp_df['q_contig'] = tmp_df['Query'].str.extract(r'([p|h][a-z]*_[^.]*).?')
    tmp_df['t_contig'] = tmp_df['Target'].str.extract(r'([p|h][a-z]*_[^.]*).?')
    #fix that if you don't extract anything return False and not 'nan'
    tmp_df['t_contig'].fillna(False, inplace=True)
    tmp_df['q_contig == t_contig'] = tmp_df.apply(lambda row: same_contig_blast(row['Query'], row['Target']), axis =1)
    tmp_df.reset_index(inplace=True, drop=True)
    blast_out_dict[tmp_key] = tmp_df.iloc[:,:]
    

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:33: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [18]:
for folder in ass_folders:
    '''
    From oriented_coords.csv files generate gff files with the following set up.
    h_contig overlap
    'query', 'ref', 'h_feature', 'query_start', 'query_end', 'query_aln_len', 'strand', 'frame', 'attribute_h'
    p_contig overlap
    'ref', 'query', 'p_feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute_p'.
    Save those file in a new folder for further downstream analysis. The same folder should include the contig and gene filtered
    gffs. 
    '''
    orient_coords_file = [os.path.join(folder, x) for x in os.listdir(folder) if x.endswith('oriented_coords.csv')][0]
    #load in df and generate several additions columns
    tmp_df = pd.read_csv(orient_coords_file, sep=',')
    #check if there is any resonable alignment if not go to the next one
    if len(tmp_df) == 0:
        print('Check on %s assembletics' % (folder))
        continue
    tmp_df['p_feature'] = "haplotig"
    tmp_df['h_feature'] ='primary_contig'
    tmp_df['strand'] = "+"
    tmp_df['frame'] = 0
    tmp_df['query_aln_len'] = abs(tmp_df['query_end']-tmp_df['query_start'])
    tmp_df['alignment_length'] = abs(tmp_df['ref_end'] - tmp_df['ref_start'])
    tmp_df.reset_index(drop=True, inplace=True)
    tmp_df.sort_values('query', inplace=True)
    tmp_df['attribute_p'] = tmp_df.apply(lambda row: att_column_p(row['query'],row['query_start'], row['query_end'],\
                                                         row['query_length'], row['query_aln_len']), axis=1)
    
    tmp_df['attribute_h'] = tmp_df.apply(lambda row: att_column_h(row['ref'], row['ref_start'], row['ref_end'],\
                                                                  row['ref_length'], row['alignment_length']), axis=1)
    
    #generate tmp gff dataframe
    tmp_p_gff_df = tmp_df.loc[:, ['ref', 'query', 'p_feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute_p']]
    tmp_p_gff_df.sort_values('ref_start',inplace=True)
    tmp_h_gff_df = tmp_df.loc[:, ['query', 'ref', 'h_feature', 'query_start', 'query_end', 'query_aln_len', 'strand', 'frame', 'attribute_h']]
    tmp_h_gff_df.sort_values('query_start', inplace=True)
    #make the outfolder and save gff files of overlap and filtered gene files
    folder_suffix = folder.split('/')[-1]
    #get the contig
    contig = re.search(r'p([a-z]*_[0-9]*)_', folder_suffix).group(1)
    out_folder = os.path.join(OUT_PATH, folder_suffix)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_name_p = os.path.join(out_folder, folder_suffix )
    tmp_p_gff_df.to_csv(out_name_p+'.p_by_h_cov.gff', sep='\t' ,header=None, index =None)
    out_name_h =  os.path.join(out_folder, folder_suffix)
    tmp_h_gff_df.to_csv(out_name_h+'.h_by_p_cov.gff' , sep='\t' ,header=None, index =None)
    #write those out to new folder for php together with the p and h gff of genes
    p_gene_gff = gene_contig_subset(p_gff, contig, 'gene')
    p_gene_gff.to_csv(out_name_p+'.p_gene.gff', sep='\t' ,header=None, index =None)
    h_gene_gff = gene_contig_subset(h_gff, contig, 'gene')
    h_gene_gff.to_csv(out_name_h+'.h_gene.gff', sep='\t' ,header=None, index =None)
    
    #next would be to do overlap between the gene gff and the corresponding alingment gff. Write this out. Dict for each gene and its corresponding h_contig/p_contig
    

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Check on /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Assembletics/Pst_104E_v12_pcontig_118_php_8kbp assembletics
Check on /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Assembletics/Pst_104E_v12_pcontig_193_php_8kbp assembletics


In [19]:
#now loop over the outfolders
out_folders = [os.path.join(OUT_PATH, x) for x in os.listdir(OUT_PATH) if x.endswith('_php_8kbp')]
out_folders.sort()

In [20]:
p_gene_h_contig_overlap_dict = {}
for out_folder in out_folders:
    tmp_folder_content = os.listdir(out_folder)
    tmp_p_gene_gff = [os.path.join(out_folder, x) for x in tmp_folder_content if x.endswith('p_gene.gff') ][0]
    tmp_p_gene_bed = pybedtools.BedTool(tmp_p_gene_gff).remove_invalid().saveas()
    tmp_p_by_h_gff = [os.path.join(out_folder, x) for x in tmp_folder_content if x.endswith('p_by_h_cov.gff') ][0]
    tmp_p_by_h_bed = pybedtools.BedTool(tmp_p_by_h_gff).remove_invalid().saveas()
    #generate a bed intersect
    p_gene_and_p_by_h = tmp_p_gene_bed.intersect(tmp_p_by_h_bed, wb=True)
    #check the length of the obtained intersect is 
    print("This is the lenght of the intersect %i" %len(p_gene_and_p_by_h ))
    if len(p_gene_and_p_by_h) == 0:
        continue
    #turn this into a dataframe 
    p_gene_and_p_by_h_df = p_gene_and_p_by_h.to_dataframe()
    p_gene_and_p_by_h_df['p_gene'] = p_gene_and_p_by_h_df[8].str.extract(r'ID=([^;]*);')
    p_gene_and_p_by_h_df['p_protein'] = p_gene_and_p_by_h_df['p_gene'].str.replace('TU', 'model')
    #safe this in the same folder as dataframe
    p_gene_and_p_by_h_df.to_csv(tmp_p_gene_gff.replace('p_gene.gff', 'gene_haplotig_intersect.df'), sep='\t', index=None)
    p_gene_and_p_by_h_df_2 = p_gene_and_p_by_h_df.loc[:, [10, 'p_protein']]
    #make a dict for the overlap of each gene with it's correspoding haplotig
    tmp_p_gene_h_overlap_df = p_gene_and_p_by_h_df_2.groupby('p_protein')[10].apply(list)
    tmp_p_gene_h_overlap_dict = dict(zip(tmp_p_gene_h_overlap_df.index, tmp_p_gene_h_overlap_df.values))
    p_gene_h_contig_overlap_dict.update(tmp_p_gene_h_overlap_dict)
    

This is the lenght of the intersect 588
This is the lenght of the intersect 548


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site

This is the lenght of the intersect 496
This is the lenght of the intersect 417


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site

This is the lenght of the intersect 319
This is the lenght of the intersect 352


...

/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site

This is the lenght of the intersect 12
This is the lenght of the intersect 7
This is the lenght of the intersect 4
This is the lenght of the intersect 0
This is the lenght of the intersect 5
This is the lenght of the intersect 0


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site

This is the lenght of the intersect 6
This is the lenght of the intersect 9


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3199: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site

In [118]:
#get out the p_protein on h_protein blast out dataframe and appand the h_contig overlap for each protein
_key = [x for x in blast_out_dict.keys() if x.startswith(p_genome) and x.split('.')[-2] == 'blastp' and x.endswith('outfmt6')][0]
p_gene_h_contig_overlap_df = pd.DataFrame([p_gene_h_contig_overlap_dict.keys(), p_gene_h_contig_overlap_dict.values()],  index = ['p_protein', 'h_contig_overlap']).T
_tmp_df = blast_out_dict[_key]
_tmp_df = _tmp_df.merge(p_gene_h_contig_overlap_df, how='outer' ,left_on='Query', right_on='p_protein')
_tmp_df['h_contig_overlap'].fillna(False, inplace=True)
#check if the protein hit resites on the overlapping haplotig
_tmp_df['t_contig == h_contig_overlap'] = _tmp_df.apply(lambda row: target_on_maped_haplotig(row['t_contig'], row['h_contig_overlap']), axis =1 )
#blast_out_dict[_key] = _tmp_df

In [119]:
#write the dataframe again
_tmp_df.to_csv(os.path.join(OUT_PATH, _key+'.allele_analysis'), index=None, sep='\t')

In [120]:
#now mangale the data a bit
#write out t_contig == h_contig_overlap best hits

In [122]:
_tmp_grouped = _tmp_df[_tmp_df['t_contig == h_contig_overlap'] == True].groupby('Query')


best_hits_t_contig_and_h_contig = _tmp_grouped.apply(lambda g: g[(g['e-value'] == g['e-value'].min())])

out_name = p_genome + '.h_contig_overlap.alleles'

best_hits_t_contig_and_h_contig.loc[:, ['Query', 'Target']].to_csv(os.path.join(ALLELE_PATH, out_name), sep='\t', index=None, header=None)

In [124]:
#now drop everything that was already captured
query_ids_to_drop = best_hits_t_contig_and_h_contig['Query'].unique().tolist()
target_ids_to_drop = best_hits_t_contig_and_h_contig['Target'].unique().tolist()

In [126]:
#get all queries that are on the respective haplotig but have not been caught before
_tmp_groupbed = _tmp_df[(~_tmp_df['Query'].isin(query_ids_to_drop))& (_tmp_df['q_contig == t_contig'] == True)].groupby('Query')
best_hits_same_contig_no_overlap = _tmp_groupbed.apply(lambda g: g[(g['e-value'] == g['e-value'].min())]) 
out_name = p_genome + '.no_specific_h_contig_overlap.alleles'
best_hits_same_contig_no_overlap.loc[:, ['Query', 'Target']].to_csv(os.path.join(ALLELE_PATH, out_name), sep='\t', index=None, header=None)

In [127]:
#now drop everything that was already captured
query_ids_to_drop = query_ids_to_drop + best_hits_same_contig_no_overlap['Query'].unique().tolist()
target_ids_to_drop = target_ids_to_drop + best_hits_same_contig_no_overlap['Target'].unique().tolist()

In [ ]:
#get all else. Meaning not on the corresponding haplotig but other haplotig

In [128]:
_tmp_groupbed = _tmp_df[(~_tmp_df['Query'].isin(query_ids_to_drop))& (_tmp_df['q_contig == t_contig'] == False) & (_tmp_df['Target'] != 'NaN' )].groupby('Query')
best_hits_different_contig = _tmp_groupbed.apply(lambda g: g[(g['e-value'] == g['e-value'].min()) ])
out_name = p_genome + '.no_respective_h_contig_overlap.alleles'
best_hits_different_contig.loc[:, ['Query', 'Target']].to_csv(os.path.join(ALLELE_PATH, out_name), sep='\t', index=None, header=None)

In [129]:
#now drop everything that was already captured
query_ids_to_drop = query_ids_to_drop + best_hits_different_contig['Query'].unique().tolist()
target_ids_to_drop = target_ids_to_drop + best_hits_different_contig['Target'].unique().tolist()

In [130]:
#no print out the p_proteins without alleles
out_name = p_genome + '.no.alleles'
_tmp_df[~_tmp_df['Query'].isin(query_ids_to_drop)].drop_duplicates(subset='Query')['Query'].to_csv(os.path.join(ALLELE_PATH, out_name), sep='\t', index=None, header=None)

### Done till here today

In [ ]:
_tmp_grouped = _tmp_df[_tmp_df['t_contig == h_contig_overlap'] == True].groupby('Query')

In [131]:
best_hits_t_contig_and_h_contig = _tmp_grouped.gr .filter(lambda g: g['e-value'] >= g['e-value'].min())

TypeError: filter function returned a Series, but expected a scalar bool

In [136]:
best_hits_t_contig_and_h_contig.reset_index(drop=True).groupby('Query').apply()

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,e-value,BitScore,QLgth,QCov,q_contig,t_contig,q_contig == t_contig,p_protein,h_contig_overlap,t_contig == h_contig_overlap
0,evm.model.pcontig_000.10,evm.model.hcontig_000_031.5,70.19,161,1,1,1,114,1,161,4e-69,207,114,141.228,pcontig_000,hcontig_000_031,True,evm.model.pcontig_000.10,"[hcontig_000_003, hcontig_000_031]",True
1,evm.model.pcontig_000.100,evm.model.hcontig_000_003.107,99.21,1141,9,0,1,1141,1,1141,0,2330,1144,99.7378,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.100,[hcontig_000_003],True
2,evm.model.pcontig_000.101,evm.model.hcontig_000_003.108,89.03,237,20,1,1,231,1,237,2e-150,421,231,102.597,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.101,[hcontig_000_003],True
3,evm.model.pcontig_000.102,evm.model.hcontig_000_003.109,100,107,0,0,1,107,1,107,7e-73,214,107,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.102,[hcontig_000_003],True
4,evm.model.pcontig_000.103,evm.model.hcontig_000_003.110,97.73,88,2,0,8,95,1,88,2e-57,174,95,92.6316,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.103,[hcontig_000_003],True
5,evm.model.pcontig_000.104,evm.model.hcontig_000_003.111,98.53,68,1,0,1,68,1,68,3e-43,136,68,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.104,[hcontig_000_003],True
6,evm.model.pcontig_000.105,evm.model.hcontig_000_003.112,100,122,0,0,1,122,1,122,1e-85,248,122,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.105,[hcontig_000_003],True
7,evm.model.pcontig_000.108,evm.model.hcontig_000_003.117,99.27,682,5,0,1,682,1,682,0,1392,682,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.108,[hcontig_000_003],True
8,evm.model.pcontig_000.109,evm.model.hcontig_000_003.117,99.27,682,5,0,1,682,1,682,0,1392,682,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.109,[hcontig_000_003],True
9,evm.model.pcontig_000.11,evm.model.hcontig_000_003.12,100,543,0,0,1,543,1,543,0,1109,543,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.11,"[hcontig_000_003, hcontig_000_031]",True


In [ ]:
tmp_blast_out

In [ ]:
tmp_p_gene_h_overlap_df = p_gene_and_p_by_h_df_2.groupby('p_protein')[10].apply(list)

tmp_p_gene_h_overlap_dict = dict(zip(p_gene_h_overlap_df.index, p_gene_h_overlap_df.values))

In [ ]:
tmp_p_gene_h_overlap_dict

In [ ]:
zip(p_gene_and_p_by_h_df_2['p_protein'], p_gene_and_p_by_h_df_2[10])

In [ ]:
tmp_blast_out[tmp_blast_out['Query'] == 'evm.model.pcontig_096.9']

In [ ]:
p_gene_and_p_by_h_df

In [ ]:
#write out the blast out dict

In [ ]:
blast_out_dict[_key].iloc[14900:14999,[0,1,-1]]

In [ ]:
p_genome

In [ ]:
p_gene_and_p_by_h = tmp_p_gene_bed.intersect(tmp_p_by_h_bed, wb=True)

In [ ]:
p_gene_and_p_by_h_df = p_gene_and_p_by_h.to_dataframe()

In [ ]:
p_gene_and_p_by_h_df['p_gene'] = p_gene_and_p_by_h_df[8].str.extract(r'ID=([^;]*);')
p_gene_and_p_by_h_df['p_protein'] = p_gene_and_p_by_h_df['p_gene'].str.replace('TU', 'model')

In [ ]:
p_gene_and_p_by_h_df.head()

In [ ]:
p_gene_and_p_by_h_df_2 = p_gene_and_p_by_h_df.loc[:, [10, 'p_protein']]

In [ ]:
p_gene_and_p_by_h_df_2

In [ ]:
contig_000_df['On p_by_h'] = contig_000_df.apply(lambda row: haplotig_checker(row['Query'], row['Target']), axis =1 )

In [ ]:
def haplotig_checker(Query, Target):
    '''Check if protein blast hit lies on the corresponding h_contig'''
    _tmp_df = p_gene_and_p_by_h_df_2[p_gene_and_p_by_h_df_2['p_protein'] == Query ]
    if Target == 'NaN':
        return False
    else:
        h_contig = re.search(r'(hcontig_[^.]*).', Target).group(1)
        if sum(_tmp_df[10].str.contains(h_contig)) > 0:
               return True
        else:
               return False

In [ ]:
contig_000_df.head()

In [ ]:
re.search(r'(hcontig[^.]*).','evm.model.hcontig_000_031.5').group(1)

In [ ]:
grouby = [(p_gene_and_p_by_h_df.groupby('p_gene').count()  > 1)]

In [ ]:
grouby #[grouby[0] == True]

In [ ]:
len(p_gene_and_p_by_h_df['p_gene'].unique())

In [ ]:
p_gene_to_haplotig_dict = p_gene_and_p_by_h_df['p']

In [ ]:
contig_000_df = blast_out_dict['Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6']

In [ ]:
contig_000_df = contig_000_df[contig_000_df['Query'].str.contains('pcontig_000')]

In [ ]:
contig_000_df.head()

In [ ]:
contig_000_df[(contig_000_df['Query'].str.contains('pcontig_000.10')) & (contig_000_df['Target'].str.contains('hcontig_000'))]

In [ ]:
def feature_specific_haplotig(x,y df):
    '''Transfers the specific haplotig that overlaps with the gene sequence'''
    

In [ ]:
#this is slow and might need to be checked. Maybe simple add a h_contig column to the blast_out_dict dataframes
def haplotig_checker(Query, Target):
    '''Check if protein blast hit lies on the corresponding h_contig. Right now assumes p_gene_and_p_by_h_df_2 exists.'''
    _tmp_df = p_gene_and_p_by_h_df_2[p_gene_and_p_by_h_df_2['p_protein'] == Query ]
    if Target == 'NaN':
        return False
    else:
        h_contig = re.search(r'(hcontig_[^.]*).', Target).group(1)
        if sum(_tmp_df[10].str.contains(h_contig)) > 0:
               return True
        else:
               return False